# Optimizing Performance (Single-Core)

## Memory allocations

Let's take a quick look at (virtual) **memory** of a process:

<br>
<img src="./imgs/stack_heap.svg" width="550">

* **Stack:**
  * very much restriced, e.g. limited size (few MB) and LIFO (last in, first out) structure
  * managed automatically
  * **fast**
* **Heap:**
  * large memory pool (many GB)
  * managed by the programmer
  * can be modified almost arbitrarily (via pointers)
  * **slow**

Allocating memory on the **heap is costly** (e.g. compared to floating point operations).

Crude benchmark:

In [ ]:
using BenchmarkTools

In [ ]:
@btime Vector{Float64}(undef, 1); # allocate single-element array

In [ ]:
@btime 1.2 + 3.4; # floating point operation

And freeing unused memory can be as well, because it triggers Julia's **garbage collector (GC)**. 

In [ ]:
@btime GC.gc();

**Performance rule: Avoid (repeated) heap allocations, especially in "hot" inner loops.**

### Mutable vs immutable types

In [ ]:
mutable struct Mutable
    x::Int64
    # ...
end

In [ ]:
m = Mutable(1)

In [ ]:
m.x = 4 # mutability

In [ ]:
function gauss_sum_mutable()
    m = Mutable(0)
    for i in 1:100_000
        m = Mutable(m.x + i)
    end
    return m
end

In [ ]:
gauss_sum_mutable()

In [ ]:
@btime gauss_sum_mutable();

(In some cases the compiler is smart enough to elide the unnecessary allocations, but not in this case.)

#### `struct` instead of `mutable struct`

In [ ]:
struct Immutable
    x::Int64
end

In [ ]:
n = Immutable(0)

In [ ]:
n.x = 4 # immutable, thus errors

In [ ]:
function gauss_sum_immutable()
    n = Immutable(0)
    for i in 1:100_000
        n = Immutable(n.x + i)
    end
    return n
end

In [ ]:
@btime gauss_sum_immutable();

This is fast! In fact, the entire computation has been "compiled away":

In [ ]:
@code_llvm debuginfo=:none gauss_sum_immutable()

Immutability is a powerful property for the compiler! It implies that **two instances with the same values are in fact identical**:

In [ ]:
Immutable(3) === Immutable(3)

In [ ]:
Mutable(3) === Mutable(3)

This means that the compiler is, for example, free to copy immutable objects for optimization, because it's impossible to distinguish the original from the copy.

**General note:**
* Immutable objects are more likely to be **stack allocated** (or even held in CPU registers only).

* Mutable objects are more likely to be allocated on the heap.

(However, these are not strict rules! Immutable objects can land on the heap and mutable ojects on the stack.)

### Beware of: "array computations"

In [ ]:
function f!(x)
    y = copy(x)
    for i in 1:100_000
        y = y + 2*y
    end
    copy!(x, y)
end

In [ ]:
@btime f!(x) setup = (x = rand(3));

* Huge number of allocations!
* Bad sign if they **scale with the number of iterations**!

#### Fix 1: Write explicit loops

In [ ]:
function f_loop!(x)
    for i in 1:100_000
        for k in eachindex(x)
            x[k] = x[k] + 2 * x[k]
        end
    end
end

@btime f_loop!(x) setup = (x = rand(3));

#### Fix 2: Broadcasting / syntactic loop fusion

In [ ]:
x = rand(3);
y = rand(3);

In [ ]:
x .* y # "element-wise" application

In [ ]:
sin(x)

In [ ]:
sin.(x) # "element-wise" application

**Also works for user-defined functions!**

In [ ]:
somefunc(x) = exp(2*x)

In [ ]:
somefunc.(x)

In [ ]:
function f_broadcast!(x)
    for i in 1:100_000
        x .= x .+ 2 .* x
        # @. x = x + 2 * x
    end
end

@btime f_broadcast!(x) setup = (x = rand(3));

Note: One also needs to broadcast the assignment (`=`) for it to be fused with the other operations.

(Recommended read: https://julialang.org/blog/2017/01/moredots/)

#### Fix 3: [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl) (if time permits)

In [ ]:
using StaticArrays

In [ ]:
sv = @SVector [1,2,3]

**Properties:**
* Size is fixed (encoded in the type)
* immutable

In [ ]:
function f_static!(x)
    @assert length(x) == 3
    s = SVector{3}(x)
    for i in 1:100_000
        s = s + 2*s
    end
    x .= s
end

In [ ]:
@btime f_static!(x) setup = (x = rand(3));

No allocations, and faster than the variants we've considered above.

### Beware of: Array slicing

By default, array-slicing creates copies!

In [ ]:
X = rand(3,3);

In [ ]:
# add up the (first three) columns of Y
add_cols(Y) = Y[:,1] .+ Y[:,2] .+ Y[:,3]

In [ ]:
@btime add_cols($X);

#### Fix: Views

In [ ]:
add_cols_views(Y) = @views Y[:,1] .+ Y[:,2] .+ Y[:,3]

@btime add_cols_views($X);

(Note that [copying data isn't always bad](https://docs.julialang.org/en/v1/manual/performance-tips/#Copying-data-is-not-always-bad))

## Memory access optimizations

### Locality

* **temporal locality**: if a memory address is accessed, there will soon be another access to that address.
* **spatial locality**: if a memory address is accessed, there will sonn be an access to a **nearby** address.

How are these locality notions reflected in hardware?

* temporal locality → **caches**
* spatial locality → memory is read in **cache lines** (multiple elements at once)

**Memory hierarchy**


<img src="./imgs/memory_hierarchy.svg" width=550px>
<br>

In [ ]:
using CpuId

In [ ]:
cpuinfo()

**Illustrative example:**
```julia
function mysum(a)
    s = zero(eltype(a))
    for i in eachindex(a)
        s = s + a[i]
    end
end
```

* `s` is repeatedly read and written → temporal locality
* `a` is accessed one element after another → spatial locality

Higher-dimensional Julia arrays are **column-major order** (like Fortran, unlinke C/C++).

<br>
<img src="./imgs/memory_order.svg" width=920px>
<br>

Let's get a feeling for the impact of (the lack of) spatial locality with a basic example.

In [ ]:
M = rand(1024,1024);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

In [ ]:
@btime fcol($M)

In [ ]:
@btime frow($M)

You can study spatial and temporal locality more deeply in the exercises (e.g. the **dMMM exercise**).

### `@inbounds`

Disables bounds checks. (Julia may segfault if you use it wrongly!)

In [ ]:
function comp()
    x = [1,2,3]
    for i in 1:100_000
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime comp();

In [ ]:
function comp_inbounds()
    x = [1,2,3]
    for i in 1:100_000
        for k in 1:3
            @inbounds x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end

@btime comp_inbounds();

# Core messages of this Notebook

* **Avoid unnecessary, repeated memory allocations.** Preallocate and/or re-use existing memory as much as possible.
* Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* Use **views** instead of copies to avoid unnecessary allocations.
* Try to make your types **immutable**, if possible.
* Be aware of spatial and temporal locality and especially **column major order** when looping over arrays.